# 助数詞（カウンター）タスクGRPO学習ノートブック（改良版）

training_counter.ipynbのログ分析結果に基づいて、報酬バランスを改善したバージョンです。

## 主な改善点
1. **報酬の厳格化**: 数字が正しくても助数詞が間違っている場合の部分点を削減
2. **カテゴリーベース評価**: 動物系（匹/羽/頭）などカテゴリー内での混同を考慮
3. **文法説明の強化**: 助数詞の使い分けに関する説明を重視
4. **数字の正確性チェック**: 漢数字・全角数字の正規化対応

## ログ分析結果
- 完全正解(3.0): 43.5%
- 部分点(0.2-2.5): 50.0%（寛大すぎる）
- 間違い(<0): 6.5%
- 平均報酬: 1.45（高すぎる）

改良版では、より厳格な評価により正しい助数詞の使用を促進します。

## 1. 環境セットアップ

In [ ]:
# GPU環境の確認
import torch
import os

print("=== GPU環境チェック ===")
print(f"CUDA利用可能: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f"GPU名: {torch.cuda.get_device_name(0)}")
    print(f"GPUメモリ: {torch.cuda.get_device_properties(0).total_memory / 1024**3:.2f} GB")
    print("✅ GPU環境が正常に検出されました！")
else:
    print("❌ GPUが検出されません！")
    print("上記の手順でGPUを有効にしてください。")
    print("その後、ランタイムを再起動してこのセルを再実行してください。")

# Colab環境かチェック
if 'COLAB_GPU' in os.environ:
    print(f"\nGoogle Colab GPU: {os.environ['COLAB_GPU']}")
elif 'COLAB_' in "".join(os.environ.keys()):
    print("\nGoogle Colab環境です。GPUを有効にしてください。")
else:
    print("\nローカル環境で実行中")

*   unsloth 2025.7.3
*   unsloth-zoo 2025.7.4

の組み合わせで動作。

https://github.com/unslothai/unsloth/issues/2983

In [ ]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth==2025.7.3 unsloth-zoo==2025.7.4 vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth==2025.7.3 unsloth-zoo==2025.7.4 vllm==0.8.5.post1

In [ ]:
#@title Colab追加インストール { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth==2025.7.3, unsloth-zoo==2025.7.4 vllm
else:
    !pip install --no-deps unsloth==2025.7.3 vllm==0.8.5.post1
    # Qwen3_(4B)_GRPO.ipynbと同じ設定
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth-zoo==2025.7.4
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer

    # vLLM requirements - vLLMはnumpyを再インストールするためColabを壊す
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

## 2. モデルのロード

In [ ]:
# GPU環境が確認できた場合のみ実行
if not torch.cuda.is_available():
    raise RuntimeError("GPUが検出されません。上記の手順でGPUを有効にしてください。")

from unsloth import FastLanguageModel
import torch

print("モデルをロード中...")
max_seq_length = 2048
lora_rank = 32

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B-Base",
    max_seq_length = max_seq_length,
    load_in_4bit = False,
    fast_inference = True,
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7,
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = lora_rank*2,
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
)

## 3.nihongo-dojoのインストール

In [ ]:
# Google Colab環境での準備
%cd /content
!unzip nihongo-dojo.zip
!pip install japanize-matplotlib scikit-learn
%cd /content/nihongo-dojo/
!pip install -e .
%cd /content/

In [ ]:
import sys, importlib
module_path = "/content/nihongo-dojo"
if module_path not in sys.path:
    sys.path.append(module_path)

import nihongo_dojo
importlib.reload(nihongo_dojo)
from nihongo_dojo.colab import TrainingLogger

## 4. チャットテンプレートの設定

In [ ]:
# チャットテンプレートを設定
system_prompt = "あなたは親切で賢いアシスタントです。日本語の助数詞を正しく使って数を数えてください。"

# デフォルト設定
reasoning_start = "<reasoning>"
reasoning_end = "</reasoning>"
solution_start = "<answer>"
solution_end = "</answer>"

chat_template = """{% if messages[0]['role'] == 'system' %}{{ messages[0]['content'] }}{% endif %}

{% for message in messages %}{% if message['role'] == 'user' %}
User: {{ message['content'] }}

{% elif message['role'] == 'assistant' %}{{ 'Assistant: ' + message['content'] }}{% endif %}{% endfor %}"""

tokenizer.chat_template = chat_template

## 5. データセットの作成

In [ ]:
# 助数詞データセット生成
import os
if not os.path.exists("./datasets/nihongo-dojo-counter/"):
    %cd /content
    !python nihongo-dojo/scripts/generate_datasets.py --tasks COUNTER_WORD --custom-size 2000 --output-format jsonl --output-dir ./datasets
else:
    print("データセットは既に生成済みです")

In [ ]:
from datasets import load_dataset, Dataset
import numpy as np
import sys

print("データセットを読み込み中...")

dataset_path = './datasets/nihongo-dojo-counter_word/'

# all.jsonlファイルが存在するか確認
import os
if os.path.exists(os.path.join(dataset_path, 'all.jsonl')):
    # 直接ファイルを読み込む方法
    import json
    data = []
    with open(os.path.join(dataset_path, 'all.jsonl'), 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    dataset = Dataset.from_list(data)
elif os.path.exists(os.path.join(dataset_path, 'train.jsonl')):
    # train.jsonlを試す
    import json
    data = []
    with open(os.path.join(dataset_path, 'train.jsonl'), 'r', encoding='utf-8') as f:
        for line in f:
            data.append(json.loads(line))
    dataset = Dataset.from_list(data)
else:
    # データセットが見つからない場合のエラーメッセージ
    raise FileNotFoundError(f"データセットが見つかりません: {dataset_path}")

print(f"データセットサイズ: {len(dataset)}")

# データセットの例を表示
print("\nデータセットの例:")
for i in range(min(3, len(dataset))):
    print(f"\n例{i+1}:")
    print(f"  問題: {dataset[i]['instruction']}{dataset[i]['input']}")
    print(f"  答え: {dataset[i]['answer']}")
    print(f"  説明: {dataset[i]['think'][:100]}...")

In [ ]:
# フォーマット変換
formatted_data = []
for item in dataset:
    question = item['instruction'] + item['input']
    answer = item['answer']
    think = item['think']
    solution = f"{reasoning_start}\n{think}\n{reasoning_end}\n{solution_start}{answer}{solution_end}"

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question},
        {"role": "assistant", "content": solution}
    ]

    formatted_data.append({
        "Messages": messages,
        "problem": question,
        "solution": solution,
        "answer": answer,
    })

dataset = Dataset.from_list(formatted_data)
print(f"フォーマット済みデータセット: {len(dataset)}個")

## 6. SFTによる事前学習（フォーマット学習）

コールドスタート問題を解消するために、`<reasoning></reasoning><answer></answer>`のフォーマットで出力ができるようにSFTで学習させます。

In [ ]:
# 短い例のみを選択
dataset = dataset.map(lambda x: {"N": len(tokenizer.apply_chat_template(x["Messages"]))})
pre_train_dataset = dataset.filter(lambda x: x["N"] <= max_seq_length/2).select(range(min(50, len(dataset))))
pre_train_dataset = pre_train_dataset.map(lambda x: {"text": tokenizer.apply_chat_template(x["Messages"], tokenize=False)})

print(f"事前学習データセット: {len(pre_train_dataset)}個")

In [ ]:
from trl import SFTTrainer, SFTConfig

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = pre_train_dataset,
    args = SFTConfig(
        dataset_text_field = "text",
        per_device_train_batch_size = 1,
        gradient_accumulation_steps = 1,
        warmup_steps = 5,
        num_train_epochs = 2,
        learning_rate = 2e-4,
        logging_steps = 5,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        report_to = "none",
    ),
)

print("フォーマット学習を開始...")
trainer.train()

## 7. ログ関連

In [ ]:
from nihongo_dojo.colab import TrainingLogger

# ログ管理インスタンスを作成（詳細ログも有効化）
# タスク名を指定してログファイル名を設定
logger = TrainingLogger(log_dir="./logs", task_name="counter", enable_detailed_logging=True)

# グローバル変数（互換性のため）
global TRAINING_LOGS, REWARD_LOGS, ACCURACY_STATS
TRAINING_LOGS = logger.training_logs
REWARD_LOGS = logger.reward_logs
ACCURACY_STATS = {
    'correct_format': [],
    'correct_answer': [],
    'partial_answer': [],
    'wrong_answer': [],
    'no_answer': []
}

## 8. 改良版GRPO報酬関数を定義

In [ ]:
# 改良版の報酬関数をインポート
from nihongo_dojo.reward import CounterRewardFunctions

# 改良版報酬関数インスタンスを作成
counter_reward_functions = CounterRewardFunctions(
    reasoning_start=reasoning_start,
    reasoning_end=reasoning_end,
    solution_start=solution_start,
    solution_end=solution_end,
    eos_token=tokenizer.eos_token
)

# バランスの取れた報酬関数を取得
balanced_rewards = counter_reward_functions.get_balanced_reward_functions()

print("改良版報酬関数:")
print("1. strict_format_check - 厳格なフォーマットチェック")
print("2. balanced_check_counter - バランスの取れた助数詞チェック")
print("3. enhanced_counter_quality - 強化された品質チェック")
print("4. check_number_accuracy - 数字正確性チェック")

In [ ]:
# nihongo_dojoライブラリのログ機能を使用
from nihongo_dojo.colab import LoggingRewardWrapper

# グローバル変数（後方互換性のため）
global PRINTED_TIMES, PRINT_EVERY_STEPS
PRINTED_TIMES = 0
PRINT_EVERY_STEPS = 5

# ログ付き報酬関数を作成（改良版を使用）
check_counter_with_logging = LoggingRewardWrapper(
    reward_func=counter_reward_functions.balanced_check_counter,
    logger=logger,
    print_every_steps=PRINT_EVERY_STEPS
)

## 9. 可視化コールバックの設定

In [ ]:
# nihongo_dojoライブラリの可視化コールバックを使用
from nihongo_dojo.colab import GRPOVisualizationCallback

# 可視化コールバックを作成
visualization_callback = GRPOVisualizationCallback(
    update_frequency=5,
    keep_history_steps=20,
    log_filename=logger.log_filename,
    logger=logger
)

## 10. GRPO学習の実行（改良版報酬関数使用）

In [ ]:
# GRPO用にフォーマット
dataset = dataset.map(lambda x: {
    "prompt": [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": x["problem"]},
    ],
    "answer": x["solution"],  # フルソリューションを保持（報酬関数側で実際の答えを抽出）
    "actual_answer": x["answer"],  # 実際の答えも保持
})

# プロンプト長でフィルタリング
tokenized = dataset.map(
    lambda x: {"tokens": tokenizer.apply_chat_template(x["prompt"], add_generation_prompt=True, tokenize=True)},
    batched=True,
)
tokenized = tokenized.map(lambda x: {"L": len(x["tokens"])})
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print(f"最大プロンプト長: {maximum_length}")

dataset = dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
print(f"フィルタ後のデータセット: {len(dataset)}個")

In [ ]:
max_prompt_length = maximum_length + 1 # + 1 念のため
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1,
    num_generations = 4,
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    max_steps = 2000,
    save_steps = 100,
    report_to = "none",
    output_dir = "outputs_counter_balanced",
)

In [ ]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        counter_reward_functions.strict_format_check,      # 厳格なフォーマットチェック
        check_counter_with_logging,                        # バランスの取れた助数詞チェック（ログ付き）
        counter_reward_functions.enhanced_counter_quality, # 強化された品質チェック
        counter_reward_functions.check_number_accuracy,    # 数字正確性チェック
    ],
    args = training_args,
    train_dataset = dataset,
    callbacks=[visualization_callback],  # ビジュアライゼーションコールバックを追加
)

print("🌸 助数詞学習のGRPO学習を開始します（改良版報酬関数使用）...")
print("📊 リアルタイムでグラフと統計情報が表示されます")
print("💡 報酬バランスが改善され、より効果的な学習が期待できます")
print("-"*80)
trainer.train()

## 11. モデルの評価

In [ ]:
# LoRAモデルを保存
model.save_lora("grpo_counter_balanced_lora")
print("改良版モデルを保存しました")

In [ ]:
# 助数詞テスト
test_questions = [
    "鳥が5いる。",
    "本を3読んだ。",
    "車が2ある。",
    "手紙を4書いた。",
    "猫が6いる。",
    "建物が7ある。",
    "紙を8使った。",
    "時計が9ある。",
    "靴を2買った。",
    "会議室は3にあります。",
]

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 1.0,
    top_k = 50,
    max_tokens = 1024,
)

print("="*80)
print("🗾 助数詞テスト（改良版モデル）")
print("="*80)

for i, question in enumerate(test_questions, 1):
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": question}
    ]

    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )

    output = model.fast_generate(
        text,
        sampling_params=sampling_params,
        lora_request=model.load_lora("grpo_counter_balanced_lora"),
    )[0].outputs[0].text

    print(f"\n{i}. 問題: {question}")
    print(f"   応答: {output}")
    print("-"*40)

## 12. 学習ログの分析

In [ ]:
# nihongo_dojoライブラリの可視化関数を使用
from nihongo_dojo.colab.visualization import plot_training_history

# 学習履歴を可視化
plot_training_history(logger.history_filename)

In [ ]:
# 拡張ログ分析
from nihongo_dojo.colab import analyze_training_logs

print("\n📊 拡張ログ分析を実行中...")
analyze_training_logs(logger.log_filename)

## 13. 改善の効果を比較

改良版の報酬関数により、以下の改善が期待されます：

1. **報酬分布の改善**: 平均報酬1.45→より厳格な評価による適正化
2. **カテゴリー認識**: 動物系（匹/羽/頭）などカテゴリー内での使い分け学習
3. **品質の向上**: 助数詞の使い分けに関する説明の充実
4. **エラーの減少**: 数字は正しいが助数詞が間違っているケースの削減

In [ ]:
# 報酬分布の分析
print("\n📊 報酬分布の分析")
print("改良版の報酬関数により、以下の変化が期待されます：")
print("\n【従来版】")
print("- 3.0 (完全正解): 43.5%")
print("- 0.5 (数字正しい・助数詞違い): 45%（寛大すぎる）")
print("- その他部分点: 5%")
print("- -1.5 (不正解): 6.5%")
print("- 平均報酬: 1.45")
print("\n【改良版（期待値）】")
print("- 2.0 (完全正解): 30-40%")
print("- 0.2 (同カテゴリー内違い): 15-25%")
print("- -0.5 (関連カテゴリー違い): 20-30%")
print("- -1.0 (無関係カテゴリー): 10-15%")
print("- -2.0 (数字も違う): 5-10%")
print("- -3.0 (フォーマットエラー): <2%")
print("\n特に、匹と羽の混同など、よくある間違いパターンに対してより適切な学習シグナルを提供します。")

## 14. モデルの保存（オプション）

In [ ]:
# 最終的なモデルをHuggingFaceに保存（オプション）
# model.push_to_hub_merged("username/grpo-counter-balanced", tokenizer, save_method="lora")
print("モデルの保存準備が完了しました。")